In [40]:
import pandas as pd
import os
import glob
try:
    code_dir
except NameError:
    code_dir = os.getcwd()
    base_dir = code_dir.replace("/codes", "")
    salmon_dir = base_dir + "/0_salmon_DE_out/1_salmon_nonTrim"
    deseq2_dir = base_dir + "/0_salmon_DE_out/2_DEseq"
    deseq2_compile_dir = base_dir + "/0_salmon_DE_out/2_DEseq_compiled"
base_dir

'/media/pipkin/ROCKET-PRO/CD8_DEV_SC/5_Chd7_KO_RNA'

In [105]:
tx2genes = '/home/pipkin/references/GRCm38.102/tx2gene.csv'
tx2genes_df = pd.read_csv(tx2genes)
tx2genes_df = tx2genes_df.set_index("GENEID")

id2gn = '/home/pipkin/references/mm_BioMart_GeneStableID_GeneName.txt'
id2gn_df = pd.read_csv(id2gn, index_col=0)

tx2gn_df = tx2genes_df.join(id2gn_df)
tx2gn_df = tx2gn_df[[True if str(x) != 'nan' else False for x in tx2gn_df['gene_name']]]
tx2gn_df = tx2gn_df.set_index('TXNAME')

# TPM

In [164]:
###----- Compile TPM and calculate average TPM
# Per transcript tpm compilation
quant_sf_files = glob.glob("%s/*/quant.sf"%salmon_dir)

tx_tpm_file = deseq2_compile_dir + "/transcript_tpm_bysample.csv"
tx_tpm_df = pd.read_csv(quant_sf_files[0], sep="\t", index_col=0)[[]]
for quant_sf_file in quant_sf_files:
    quant_sf_sp = quant_sf_file.split("/")[-2]
    quant_sf_sp = quant_sf_sp.replace("naive","Naive")
    quant_sf_df = pd.read_csv(quant_sf_file, sep="\t", index_col=0)[['TPM']]
    quant_sf_df.columns = [quant_sf_sp]
    tx_tpm_df = tx_tpm_df.join(quant_sf_df)
tx_tpm_df.index = [x.split(".")[0] for x in tx_tpm_df.index]
tx_tpm_df_gn = tx2gn_df.join(tx_tpm_df, how='inner')
tx_tpm_df_gn.to_csv(tx_tpm_file)

# Per gene tpm
gn_tpm_file = deseq2_compile_dir + "/gene_tpm_bysample.csv"
tx_tpm_df_gn = tx2gn_df.join(tx_tpm_df, how='inner')
gn_tpm_df = tx_tpm_df_gn.groupby("gene_name").sum()
gn_tpm_df.to_csv(gn_tpm_file)

# Per condition average - transcript tpm
tx_tpm_file = deseq2_compile_dir + "/transcript_tpm.csv"
tx_tpm_df_t = tx_tpm_df.copy().transpose()
tx_tpm_df_t['cond'] = ["_".join(x.split("_")[:-1]) for x in tx_tpm_df_t.index]
tx_tpm_cond_df = tx_tpm_df_t.groupby("cond").mean().transpose()
tx_tpm_cond_df.columns.name = ''
tx_tpm_cond_df_gn = tx2gn_df.join(tx_tpm_cond_df, how='inner')
tx_tpm_cond_df_gn.to_csv(tx_tpm_file)

# Per condition average - gene tpm
gn_tpm_file = deseq2_compile_dir + "/gene_tpm.csv"
gn_tpm_df_t = gn_tpm_df.copy().transpose()
gn_tpm_df_t['cond'] = ["_".join(x.split("_")[:-1]) for x in gn_tpm_df_t.index]
gn_tpm_cond_df = gn_tpm_df_t.groupby("cond").mean().transpose()
gn_tpm_cond_df.columns.name = ''
gn_tpm_cond_df.to_csv(gn_tpm_file)

# Normalized counts

In [20]:
###----- Calculate averge normalized count
norm_count_file = deseq2_compile_dir + "/DESeq2_normalized_counts_bysample_gn.csv"
avg_norm_count_file = deseq2_compile_dir + "/DESeq2_normalized_counts_gn.csv"

norm_count_df = pd.read_csv(norm_count_file, index_col=0)
all_genes = norm_count_df.index.tolist()

# Create empty df for average normalized count
avg_norm_df = pd.DataFrame({"gene_name": all_genes}).set_index("gene_name")
conds = ["_".join(x.split("_")[:-1]) for x in norm_count_df.columns]
# Calculate averge for each condition    
for cond in conds:
    cond_reps = [x for x in norm_count_df.columns if cond+"_" in x]
    cond_df = norm_count_df[cond_reps]
    avg_norm_df[cond] = cond_df.mean(axis=1).tolist()
avg_norm_df.to_csv(avg_norm_count_file)

# Compile DESEQ outputs

In [21]:
###----- Compile DEseq output
deseq2_files = glob.glob("%s/gene_name/*.csv"%deseq2_dir)
all_genes = pd.read_csv(deseq2_files[0])['gene_name']
all_genes = [x for x in all_genes if str(x) != 'nan']
all_genes = list(set(all_genes))

In [39]:
for i in ['log2FoldChange','stat', 'pvalue', 'padj']:
    i_df = pd.DataFrame({"gene_name": all_genes}).set_index("gene_name")
    for deseq2_file in deseq2_files:
        # Label name
        deseq2_file_name_simp = deseq2_file.split("/")[-1].replace(".csv","").replace("-Salmon_gn","")
        # Uniform dataframe
        deseq2_df = pd.read_csv(deseq2_file)
        deseq2_df.columns = ["gene_name"] + deseq2_df.columns.tolist()[1:]
        deseq2_i_df = deseq2_df[['gene_name', i]]
        deseq2_i_df.columns = ['gene_name', deseq2_file_name_simp]
        # Remove none gene name match
        deseq2_i_df =  deseq2_i_df[[True if str(x) != 'nan' else False for x in deseq2_i_df['gene_name']]]
        # Remove duplicated genes
        deseq2_i_df = deseq2_i_df.drop_duplicates(subset=['gene_name'])
        # Reduce float point precision
        deseq2_i_df = deseq2_i_df.round(3)
        deseq2_i_df = deseq2_i_df.set_index("gene_name")

        i_df = i_df.join(deseq2_i_df)
        del deseq2_df
        del deseq2_i_df
    i_df_name = deseq2_compile_dir + "/DESeq2_%s.csv"%i
    i_df.to_csv(i_df_name)